In [1]:
import cftime
import csv
import datetime
import numpy as np
import pandas as pd

In [7]:
expname = "odiv-251"

create_csv(expname)
create_static_storms(expname)

In [3]:
def create_csv(expname):

    pfile_in  = f"/work/Katherine.Turner/tempest-extremes/{expname}/node_files/SH_SN.txt"
    pfile_out = f"/work/Katherine.Turner/tempest-extremes/{expname}/node_files/SH_SN.csv"

    event = 0

    # Open input and output files
    with open(pfile_in, "r") as file_in, open(pfile_out, "w", newline="") as file_out:
        reader = csv.reader(file_in, delimiter="\t")
        writer = csv.writer(file_out, delimiter="\t")

        # Write header for the output file
        writer.writerow(["Event", "Year", "Month", "Day", "Time", "Lon", "Lat", "SLP", "Windspeed"])

        # Iterate over each line in the input file
        for line in reader:
            # Check if the line contains date information
            if len(line) < 7:
                event = event + 1  # Update event number
            if len(line) >= 7: 
                slp, wind, yr, mo, day, time = line[-6:]
                lon, lat = line[3:5]
                writer.writerow([event, yr, mo, day, time, lon, lat, slp, wind])

In [6]:
def create_static_storms(expname):
    file = open(f"/work/Katherine.Turner/tempest-extremes/{expname}/node_files/SH_SN.txt", 'r')
    lines = file.readlines()
    
    master_list = []
    storm_entry = []
    for line in lines:
        if line.startswith("start"):
            master_list.append(storm_entry)
            storm_entry = []
        else:
            storm_entry.append(line)
            
    master_list = [x for x in master_list if len(x) > 0]
    static_master = [process_storm(x) for x in master_list]
    
    output_file_static = f"/work/Katherine.Turner/tempest-extremes/{expname}/node_files/SH_static.txt"
    
    # create limits for time period so that we don't have static storms that start before or end after the available data
    time0 = int(master_list[0][0].replace("\n","").split("\t")[-4])
    time1 = int(master_list[-1][0].replace("\n","").split("\t")[-4])

    with open(output_file_static, 'w') as out_file:
        for storm in static_master:
            if ((storm[0][-4] >= time0) & (storm[-1][-4] <= time1)):
                storm_header = "start\t11" + str(storm[0][-4:]).replace("(", "\t").replace(", ","\t").replace(")", "\n")
                out_file.write(storm_header)
                for storm_line in storm:
                    line_data = str(storm_line).replace("(", "\t").replace(", ","\t").replace(")", "\n")
                    out_file.write(line_data)

In [5]:
def process_storm(timelist):
    df = pd.DataFrame([x.replace("\n","").split("\t") for x in timelist if len(x)],
                      columns=["null","i","j","lon","lat","mslp","maxwind","year","month","day","hour"])
    
    rowmin = df[df.mslp == df.mslp.min()]
    
    if len(rowmin) > 1:
        rowmin = rowmin.iloc[[0]]
    
    i = int(rowmin.i.values)
    j = int(rowmin.j.values)
    lat = float(rowmin.lat.values)
    lon = float(rowmin.lon.values)
    mslp = float(rowmin.mslp.values)
    maxwind = float(rowmin.maxwind.values)

    tobj = cftime.DatetimeNoLeap(int(rowmin.year),int(rowmin.month),int(rowmin.day),int(rowmin.hour))

    times = np.arange(0,11)
    times = [tobj + datetime.timedelta(days=int(x)) for x in times]

    new_entries = [(i,j,lon,lat,mslp,maxwind,int(x.strftime("%Y")),int(x.strftime("%m")),int(x.strftime("%d")),int(x.strftime("%H"))) for x in times]
    
    return new_entries